# Segmenting and Clustering Neighborhoods in Toronto
## Exercise 1 - Creating a data frame from Wikipedia

### Import libraries

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import urllib.request # import the library we use to open URLs

from bs4 import BeautifulSoup  # import the BeautifulSoup library so we can parse HTML and XML documents

#!pip install geocoder
import geocoder # import geocoder

import requests # library to handle requests

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json # library to handle JSON files

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!pip install folium 
import folium # map rendering library

import numpy as np

print("Imported libraries.")

Imported libraries.


### Download and explore dataset

In [28]:
# I have followed the tutorial from: https://simpleanalytical.com/how-to-web-scrape-wikipedia-python-urllib-beautiful-soup-pandas
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page, 'lxml')

The table we want is under the tag "wikitable sortable". The code below returns all instances of the table tag.

In [30]:
# use the 'find_all' function to bring back all instances of the 'table' tag in the HTML and store in 'all_tables' variable
all_tables = soup.find_all("table")

The code below returns the table we want:

In [31]:
right_table = soup.find('table', class_='wikitable sortable')

Now I'll just iterate over the rows of the table and append results to lists:

In [5]:
A=[]
B=[]
C=[]

for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==3 and cells[1].find(text=True).rstrip() != 'Not assigned': # Here I am ignoring rows for which Borough == 'Not assigned'. I have to strip the 
        A.append(cells[0].find(text=True).rstrip())
        B.append(cells[1].find(text=True).rstrip())
        C.append(cells[2].find(text=True).rstrip())

Lastly, I will create a data frame with pandas:

In [6]:
codes = pd.DataFrame(A, columns = ['PostalCode'])
codes['Borough'] = B
codes['Neighborhood'] = C
codes.shape

(103, 3)

## Exercise 2 - Get geo-location data for the locations

In [7]:
# loop until you get the coordinates
def get_latlong(postal_code):
    # initialize your variable to None
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
    return(lat_lng_coords)

latitude = [] # need to initiate the lists
longitude = []

# Iterate over postal codes
for idx in codes.index:
    latitude.append(get_latlong(codes['PostalCode'][idx])[0])
    longitude.append(get_latlong(codes['PostalCode'][idx])[1])

In [8]:
codes['Latitude'] = latitude
codes['Longitude'] = longitude
codes.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.650964,-79.353041
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.661790,-79.389390


## Exercise 3 - Exploring and clustering the neighborhoods in Toronto

### 1. Explore Neighborhoods in Toronto

#### First I will subset the data frame to explore the North York borough

In [9]:
north_york = codes[codes['Borough'] == 'North York'].reset_index(drop=True)
north_york.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.752935,-79.335641
1,M4A,North York,Victoria Village,43.728102,-79.311890
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211
3,M3B,North York,Don Mills,43.748900,-79.357220
4,M6B,North York,Glencairn,43.707279,-79.447500


#### After defining the Forsquare Credentials and Version I will get all the nearby venues in each neighborhood in North York

In [10]:
# @hidden_cell
CLIENT_ID = 'Q0UUCAFT5ABUE0L5WW3K15JEFUFPCQ3L0KF0CPUFEFMX4EPX' # your Foursquare ID
CLIENT_SECRET = 'MIFAYELUPRVBCQKQ5RF2W3ZL00O2AYUQYLTLPF4VLCK05UCK' # your Foursquare Secret
ACCESS_TOKEN = '1LRLM5Y31UPSKL4C1DPA0KWBTSPTHEXWFMOOFINPPTNVWNBW'

In [11]:
VERSION = '20180605' # Foursquare API version
radius = 500
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&oauth_token={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            ACCESS_TOKEN,
            VERSION, 
            lat, 
            lng, 
            radius,
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

north_york_venues = getNearbyVenues(names = north_york['Neighborhood'],
                                   latitudes = north_york['Latitude'],
                                   longitudes = north_york['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor, Lawrence Heights
Don Mills
Glencairn
Don Mills
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Fairview, Henry Farm, Oriole
Northwood Park, York University
Bayview Village
Downsview
York Mills, Silver Hills
Downsview
North Park, Maple Leaf Park, Upwood Park
Humber Summit
Willowdale, Newtonbrook
Downsview
Bedford Park, Lawrence Manor East
Humberlea, Emery
Willowdale, Willowdale East
Downsview
York Mills West
Willowdale, Willowdale West


In [12]:
print(north_york_venues.shape)
north_york_venues.head()

(526, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.752935,-79.335641,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.752935,-79.335641,GTA Restoration,43.753396,-79.333477,Fireworks Store
2,Parkwoods,43.752935,-79.335641,Corrosion Service Company Limited,43.752432,-79.334661,Construction & Landscaping
3,Parkwoods,43.752935,-79.335641,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Parkwoods,43.752935,-79.335641,Three Valleys Park,43.751195,-79.337356,Park


I'll check how many venues were returned for each neighborhood:

In [13]:
north_york_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Wilson Heights, Downsview North",32,32,32,32,32,32
Bayview Village,3,3,3,3,3,3
"Bedford Park, Lawrence Manor East",42,42,42,42,42,42
Don Mills,23,23,23,23,23,23
Downsview,41,41,41,41,41,41
"Fairview, Henry Farm, Oriole",100,100,100,100,100,100
Glencairn,19,19,19,19,19,19
Hillcrest Village,2,2,2,2,2,2
Humber Summit,4,4,4,4,4,4


I'll find out how many unique categories can be curated from all the returned venues:

In [14]:
print('There are {} uniques categories.'.format(len(north_york_venues['Venue Category'].unique())))

There are 143 uniques categories.


### 2. Analyze each Neighborhood in North York

In [15]:
# one hot encoding
north_york_onehot = pd.get_dummies(north_york_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
north_york_onehot['Neighborhood'] = north_york_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [north_york_onehot.columns[-1]] + list(north_york_onehot.columns[:-1])
north_york_onehot = north_york_onehot[fixed_columns]

north_york_onehot.head()

,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Camera Store,Caribbean Restaurant,Child Care Service,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Financial or Legal Service,Fireworks Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lawyer,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Music Store,Optical Shop,Other Great Outdoors,Park,Pastry Shop,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
print(north_york_onehot.shape)

(526, 144)


Next, I'll group rows by neighborhood and by taking the mean of the frequency of occurrence of each category:

In [17]:
north_york_grouped = north_york_onehot.groupby('Neighborhood').mean().reset_index()
north_york_grouped

,Neighborhood,ATM,Accessories Store,African Restaurant,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,Baby Store,Bakery,Bank,Bar,Baseball Field,Basketball Court,Beer Store,Bookstore,Boutique,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Stop,Business Service,Butcher,Café,Camera Store,Caribbean Restaurant,Child Care Service,Chinese Restaurant,Chocolate Shop,Clothing Store,Coffee Shop,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Electronics Store,Falafel Restaurant,Fast Food Restaurant,Financial or Legal Service,Fireworks Store,Flower Shop,Food & Drink Shop,Food Court,Food Service,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gas Station,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Health & Beauty Service,Health Food Store,Hobby Shop,Home Service,Hotel,IT Services,Ice Cream Shop,Indian Restaurant,Indonesian Restaurant,Insurance Office,Intersection,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Lawyer,Leather Goods Store,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Movie Theater,Moving Target,Music Store,Optical Shop,Other Great Outdoors,Park,Pastry Shop,Pet Store,Pharmacy,Piano Bar,Pizza Place,Playground,Plaza,Pool,Portuguese Restaurant,Post Office,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Sandwich Place,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Snack Place,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Trail,Video Game Store,Vietnamese Restaurant,Women's Store,Yoga Studio
0,"Bathurst Manor, Wilson Heights, Downsview North",0.00,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.031250,0.0,0.000000,0.062500,0.000000,0.0000,0.000000,0.00000,0.00,0.00000,0.00000,0.03125,0.000000,0.000000,0.00,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.00000,0.03125,0.00,0.00,0.062500,0.00000,0.000000,0.031250,0.00000,0.00000,0.031250,0.000000,0.000000,0.03125,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.00000,0.000000,0.000,0.03125,0.03125,0.000000,0.031250,0.03125,0.00000,0.031250,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.062500,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.031250,0.00,0.062500,0.000000,0.00000,0.000000,0.000000,0.000000,0.031250,0.000000,0.031250,0.062500,0.0,0.031250,0.000000,0.000000,0.000000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.0,0.031250,0.031250,0.000000,0.00,0.031250,0.00,0.00,0.062500,0.00,0.000000,0.00000,0.000000,0.03125,0.00,0.031250,0.00,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.00,0.00
1,Bayview Village,0.00,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0000,0.000000,0.00000,0.00,0.00000,0.00000,0.00000,0.000000,0.000000,0.00,0.00000,0.000000,0.00000,0.000000,0.00,0.000000,0.00000,0.00000,0.00,0.00,0.000000,0.00000,0.333333,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.0,0.333333,0.0,0.00000,0.000000,0.000,0.00000,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.00000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.00000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000

In [18]:
# confirming the new size
print(north_york_grouped.shape)

(20, 144)


Now, I'll print each neighborhood along with the top 5 most common venues:

In [19]:
num_top_venues = 5

for hood in north_york_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_grouped[north_york_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0        Coffee Shop  0.06
1           Pharmacy  0.06
2  Mobile Phone Shop  0.06
3     Ice Cream Shop  0.06
4                Spa  0.06


----Bayview Village----
                        venue  freq
0  Construction & Landscaping  0.33
1                       Trail  0.33
2                 Flower Shop  0.33
3                         ATM  0.00
4               Movie Theater  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0                 Spa  0.07
1  Italian Restaurant  0.07
2           Juice Bar  0.05
3          Hobby Shop  0.05
4      Sandwich Place  0.05


----Don Mills----
                        venue  freq
0                       Trail  0.13
1          Athletics & Sports  0.13
2  Construction & Landscaping  0.09
3                  Restaurant  0.09
4                Home Service  0.09


----Downsview----
                   venue  freq
0          Grocery Store  0.07
1         

We can also display this information in a pandas data frame:

In [20]:
# creating the new dataframe and display the top 10 venues for each neighborhood.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = north_york_grouped['Neighborhood']

for ind in np.arange(north_york_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(north_york_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Wilson Heights, Downsview North",Mobile Phone Shop,Spa,Coffee Shop,Ice Cream Shop,Pharmacy,Bank,Convenience Store,Chinese Restaurant,Deli / Bodega,Pet Store
1,Bayview Village,Construction & Landscaping,Flower Shop,Trail,Distribution Center,Food & Drink Shop,Fireworks Store,Financial or Legal Service,Fast Food Restaurant,Falafel Restaurant,Electronics Store
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Spa,Sandwich Place,Mobile Phone Shop,Hobby Shop,Coffee Shop,Juice Bar,Thai Restaurant,Sushi Restaurant,Pizza Place
3,Don Mills,Trail,Athletics & Sports,Home Service,Construction & Landscaping,Restaurant,Coffee Shop,Mattress Store,Spa,Furniture / Home Store,Jewelry Store
4,Downsview,Mobile Phone Shop,Grocery Store,Home Service,Discount Store,Business Service,Pizza Place,Coffee Shop,Vietnamese Restaurant,Shopping Mall,Pharmacy


## 4. Clustering neighborhoods in North York

Run *k*-means to cluster the neighborhood into 3 clusters:

In [21]:
# set number of clusters
kclusters = 3

north_york_grouped_clustering = north_york_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(north_york_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

north_york_merged = north_york

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
north_york_merged = north_york_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_york_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.752935,-79.335641,1,Construction & Landscaping,Park,Convenience Store,Fireworks Store,Bus Stop,Food & Drink Shop,Dessert Shop,Diner,Discount Store,Department Store
1,M4A,North York,Victoria Village,43.728102,-79.311890,0,Coffee Shop,French Restaurant,Pizza Place,Portuguese Restaurant,Financial or Legal Service,Bridal Shop,Park,Intersection,Gym,Fast Food Restaurant
2,M6A,North York,"Lawrence Manor, Lawrence Heights",43.723265,-79.451211,0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Cosmetics Shop,Furniture / Home Store,Boutique,Sandwich Place,American Restaurant,Sushi Restaurant
3,M3B,North York,Don Mills,43.748900,-79.357220,0,Trail,Athletics & Sports,Home Service,Construction & Landscaping,Restaurant,Coffee Shop,Mattress Store,Spa,Furniture / Home Store,Jewelry Store
4,M6B,North York,Glencairn,43.707279,-79.447500,0,Spa,Pizza Place,Mediterranean Restaurant,Business Service,Rental Car Location,Italian Restaurant,Pharmacy,Sushi Restaurant,Japanese Restaurant,Asian Restaurant


Finally, I'll visualize the resulting clusters:

In [22]:
address = 'North York, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of North York are {}, {}.'.format(latitude, longitude))
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_merged['Latitude'], north_york_merged['Longitude'], north_york_merged['Neighborhood'], north_york_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

The geograpical coordinates of North York are 43.7543263, -79.44911696639593.


## 5. Examine Clusters

In [23]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 0, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Coffee Shop,French Restaurant,Pizza Place,Portuguese Restaurant,Financial or Legal Service,Bridal Shop,Park,Intersection,Gym,Fast Food Restaurant
2,North York,0,Clothing Store,Coffee Shop,Women's Store,Restaurant,Cosmetics Shop,Furniture / Home Store,Boutique,Sandwich Place,American Restaurant,Sushi Restaurant
3,North York,0,Trail,Athletics & Sports,Home Service,Construction & Landscaping,Restaurant,Coffee Shop,Mattress Store,Spa,Furniture / Home Store,Jewelry Store
4,North York,0,Spa,Pizza Place,Mediterranean Restaurant,Business Service,Rental Car Location,Italian Restaurant,Pharmacy,Sushi Restaurant,Japanese Restaurant,Asian Restaurant
5,North York,0,Trail,Athletics & Sports,Home Service,Construction & Landscaping,Restaurant,Coffee Shop,Mattress Store,Spa,Furniture / Home Store,Jewelry Store
7,North York,0,Mobile Phone Shop,Spa,Coffee Shop,Ice Cream Shop,Pharmacy,Bank,Convenience Store,Chinese Restaurant,Deli / Bodega,Pet Store
8,North York,0,Clothing Store,Shoe Store,Women's Store,Fast Food Restaurant,Restaurant,Electronics Store,Cosmetics Shop,Coffee Shop,Miscellaneous Shop,Jewelry Store
9,North York,0,Spa,Pizza Place,Furniture / Home Store,Massage Studio,Restaurant,Fast Food Restaurant,Lawyer,Financial or Legal Service,Rental Car Location,Market
11,North York,0,Mobile Phone Shop,Grocery Store,Home Service,Discount Store,Business Service,Pizza Place,Coffee Shop,Vietnamese Restaurant,Shopping Mall,Pharmacy
12,North York,0,Coffee Shop,Bank,Butcher,Baseball Field,Ice Cream Shop,Sandwich Place,Park,Cosmetics Shop,Bus Stop,Shopping Mall


In [24]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 1, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,1,Construction & Landscaping,Park,Convenience Store,Fireworks Store,Bus Stop,Food & Drink Shop,Dessert Shop,Diner,Discount Store,Department Store
10,North York,1,Construction & Landscaping,Flower Shop,Trail,Distribution Center,Food & Drink Shop,Fireworks Store,Financial or Legal Service,Fast Food Restaurant,Falafel Restaurant,Electronics Store
19,North York,1,Park,Playground,Construction & Landscaping,Yoga Studio,Distribution Center,Fireworks Store,Financial or Legal Service,Fast Food Restaurant,Falafel Restaurant,Electronics Store
22,North York,1,Construction & Landscaping,Flower Shop,Park,Electronics Store,Convenience Store,Health Food Store,Fireworks Store,Cosmetics Shop,Hotel,Cupcake Shop


In [25]:
north_york_merged.loc[north_york_merged['Cluster Labels'] == 2, north_york_merged.columns[[1] + list(range(5, north_york_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,North York,2,Park,Residential Building (Apartment / Condo),Yoga Studio,Distribution Center,Fireworks Store,Financial or Legal Service,Fast Food Restaurant,Falafel Restaurant,Electronics Store,Discount Store


Observations: It looks like most of the neighborhoods in cluster 0 fall into the residential category (with lots of coffee shops, clothing stores, etc), whereas cluster 1 contains nrighboords that are still under construction. Cluster 2 contains just one neighborhood. 